## Compilation basics

- Use `g++` (or `clang++`) so the C++ standard library is linked automatically;
- `gcc` would require `-lstdc++`.
- Keep headers in `.hpp` and implementations in `.cpp` files.
- Typical one-file build while experimenting:

```bash
g++ main.cpp -std=c++17 -Wall -Wextra -Wconversion -o app
```


## Using C libraries

- Wrap C headers with `extern "C"` so C++ name mangling does not break linking.
- Use `#ifdef __cplusplus` guards to keep the header usable from both C and C++.

```cpp
#ifdef __cplusplus
extern "C" {
#endif

void my_c_function();

#ifdef __cplusplus
}
#endif
```

## Build System
- Use CMake to generate build files and keep sources clean with an out-of-source `build/` directory.
- `CMakeLists.txt` declares targets and dependencies; `CMakePresets.json` stores configure presets for debug/release.
- Point your editor (e.g. `.vscode/settings.json`) at the preset include paths so IntelliSense matches the build.

```text
calculator_project/
├── CMakeLists.txt       <-- Build targets and dependencies
├── CMakePresets.json    <-- Configure presets and build directories
├── include/
│   └── simple_math.hpp  <-- The Declaration (.hpp)
├── src/
│   ├── simple_math.cpp  <-- The Implementation (.cpp)
│   └── main.cpp         <-- The Entry Point
└── build/               <-- Out-of-source build output
```

## Including External Libraries

- Declare external dependencies in `CMakeLists.txt` (e.g., `FetchContent` + `target_link_libraries`).
- Once headers are on the include path, include them with angle brackets: `#include <lib/header.hpp>`.
- Let CMake fetch/build during configure so the repo stays source-only.


## Namespaces & Scopes

- Namespaces group related code and prevent symbol collisions across libraries.
- Prefer small, descriptive namespaces and avoid `using namespace std;` in headers.


In [1]:
#include <iostream>
#include <memory>
#include <cstring>
#include <string>
#include <vector>

In [2]:
namespace Graphics {
    struct Data {
        int id;
    };

    void init() {
        std::cout << "Initializing Graphics Engine..." << std::endl;
    }
}

namespace Audio {
    void init() {
        std::cout << "Initializing Audio System..." << std::endl;
    }
    struct Data {
        float value;
    };
}

## Value initialization

- We can initialize values using lists, for example:

```cpp
struct User {
    int id;
    std::string name;
    std::string email;
};

User u1{42, "Ada Lovelace", "ada@example.com"}; // id, name, email
```

- Works for aggregates or constructors that accept those arguments.
- Brace initialization also prevents narrowing conversions (`int i{3.14}; // error`).


In [3]:
{
    Graphics::init();
    Audio::init();

    Graphics::Data gfxData{1};
    // Audio::Data audioData{0.5f};
    Audio::Data audioData;
    audioData.value = 0.5f;

    std::cout << "Graphics Data ID: " << gfxData.id << std::endl;
    std::cout << "Audio Data Value: " << audioData.value << std::endl;
}

Initializing Graphics Engine...
Initializing Audio System...
Graphics Data ID: 1
Audio Data Value: 0.5


## Using the heap to allocate structs

- The snippets above allocate on the stack; use the heap when an object must outlive the current scope.
- `auto` lets the compiler deduce pointer types so the line focuses on intent.


In [ ]:
{
    auto* u = new Audio::Data{0.75f};
    std::cout << "Dynamically allocated Audio Data Value: " << u->value << std::endl;
    delete u;
}

Dynamically allocated Audio Data Value: 0.75


- Prefer `std::make_unique` so the pointer frees itself when it goes out of scope.


In [5]:
{
    auto u = std::make_unique<Audio::Data>(Audio::Data{0.9f});
    std::cout << "Unique Pointer Audio Data Value: " << u->value << std::endl;
}

Unique Pointer Audio Data Value: 0.9


## How to zero out data

- `std::memset` works for plain-old-data; using it on objects with invariants (like `std::string`) corrupts them.
- Reset C++ objects by assigning a value-initialized instance instead of zeroing raw memory.


In [3]:
namespace C_Style_vs_CPP {
    struct C_Style_Struct {
        int id;
        float value;
    };

    struct Cpp_Class {
        int id;
        std::string name;
    };
}

In [7]:
{
    using namespace C_Style_vs_CPP;
    // --- SAFE ---
    C_Style_Struct c_obj;
    std::memset(&c_obj, 0, sizeof(c_obj)); // Totally fine for Plain Old Data (POD) types
    std::cout << "memset on POD: Safe." << std::endl;

    // --- DANGEROUS ---
    Cpp_Class cpp_obj;
    cpp_obj.name = "Hello";

    // UNCOMMENTING THE LINE BELOW WOULD CORRUPT THE STRING INTERNAL POINTERS
    // std::memset(&cpp_obj, 0, sizeof(cpp_obj));

    // In C++, we use Constructors or assignment to reset objects.
    cpp_obj = Cpp_Class{}; // Reassign a fresh empty object

    std::cout << "Reset C++ object safely." << std::endl;
}

memset on POD: Safe.
Reset C++ object safely.


In [ ]:
{
    using namespace C_Style_vs_CPP;
    auto ptr = std::make_unique<Cpp_Class>();
    ptr->name = "Unique Pointer Example";
    ptr->id = 42;

    // the line below assigns a new Cpp_Class object to the memory location pointed by ptr
    *ptr = Cpp_Class{};
}

## C++ assignment copies data, not just pointers

In C, assigning one pointer to another reuses the same allocation:

```c
char * a = (char*) malloc(40);
strcpy(a, "tchucunarublaze");

char * b = (char*) malloc(40);
strcpy(b, "marolivez");

free(a);
a = b;

printf("%s\n", a);

// we must free *a and b only once*
free(a);
```

The address stored in `b` is shallow-copied into `a`, so both names refer to the same allocation.

### C++ assignment allocates new storage and copies the elements.


In [9]:
{
    std::vector<int> vec = {1, 2, 3, 4, 5};
    std::vector<int> vec_copy = vec; // Deep copy
    vec_copy[0] = 10;

    std::cout << "Original variable vector adress: " << &vec << std::endl;
    std::cout << "Original vector heap data adress: " << vec.data() << std::endl;
    std::cout << "Copied variable vector adress: " << &vec_copy << std::endl;
    std::cout << "Copied vector heap data adress: " << vec_copy.data() << std::endl;
}

Original variable vector adress: 0x7ffe14fc27c0
Original vector heap data adress: 0x5754bd885720
Copied variable vector adress: 0x7ffe14fc2760
Copied vector heap data adress: 0x5754bf92cc30


### What to do

1. `T dest = src;` makes a deep copy of the underlying data.
2. Use `const T&` for function arguments when you want to read data without copying.
3. Use `T&` when you want to modify data in place.
4. Use `T*` only when `nullptr` is a valid "no object" state, and be clear about ownership.


In [4]:
namespace Functions {
    using namespace C_Style_vs_CPP;
    void print_user(const Cpp_Class& u) {          // read-only: no copy
        std::cout << "id=" << u.id << ", name=" << u.name << '\n';
    }

    void set_name(Cpp_Class& u, const std::string& n) { // modify in place: no copy
        u.name = n;
    }

    void maybe_clear(Cpp_Class* u) {               // pointer allows “no object”
        if (!u) return;
        *u = Cpp_Class{}; // assign new value (deep copy of a temporary)
    }
}


In [ ]:
{
    using namespace Functions;
    Cpp_Class a{1, "Alice"};
    Cpp_Class b = a;          // deep copy (assignment/copy-ctor)
    b.id = 2;                 // prove they’re independent

    print_user(a);            // const T& (no copy)
    print_user(b);

    set_name(a, "Alice Updated"); // T& (in-place modification)
    // a.name = "AliceUpdated"; //Direct modification will also work
    print_user(a);

    maybe_clear(nullptr);     // safe to call with no object
    maybe_clear(&b);          // pointer form; clears b’s data
    print_user(b);
}

id=1, name=Alice
id=2, name=Alice
id=1, name=Bruna
id=0, name=


In [5]:
{
    using namespace Functions;
    auto u = std::make_unique<Cpp_Class>();
    u->name = "Unique Pointer Example";
    u->id = 42;

    // Dereference unique_ptr to pass to function
    print_user(*u);
}

id=42, name=Unique Pointer Example
